# Cuaderno Clase PLN - Representación de Texto (BoW y TF-IDF)

**Objetivos:**
*   Repasar el concepto de Bag-of-Words (BoW).
*   Implementar BoW usando `CountVectorizer` de Scikit-learn.
*   Entender la necesidad de ponderar palabras: Introducción a TF-IDF.
*   Implementar TF-IDF usando `TfidfVectorizer` de Scikit-learn.
*   Comparar las representaciones BoW y TF-IDF.
*   Reflexionar sobre las limitaciones y próximos pasos.

**Agenda:**
1.  Instalaciones e Importaciones
2.  Preparación de Datos: Preprocesamiento (Reutilizando lo del martes)
3.  Bag-of-Words (BoW) con Scikit-learn (`CountVectorizer`)
4.  ¿Por qué BoW no es suficiente? Introducción a TF-IDF
5.  TF-IDF con Scikit-learn (`TfidfVectorizer`)
6.  Comparando BoW y TF-IDF
7.  Micro-Laboratorio (Ejercicio Práctico)
8.  Brainstorming

# 1. Instalaciones e Importaciones

In [1]:
# Instalar si es necesario (sklearn suele venir en Colab)
# !pip install scikit-learn nltk spacy pandas > /dev/null # Pandas para mostrar matrices fácil
# !python -m spacy download es_core_news_sm > /dev/null

# Importaciones
import nltk
import spacy
import re
import string
import pandas as pd # Para DataFrames

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [3]:
!python -m spacy download es_core_news_sm > /dev/null
nlp = spacy.load('es_core_news_sm')
print("Modelo de spaCy 'es_core_news_sm' cargado.")

Modelo de spaCy 'es_core_news_sm' cargado.


In [4]:
nltk.download('stopwords')
stopwords_es = nltk.corpus.stopwords.words('spanish')

print(f"\nUsando {len(stopwords_es)} stopwords de NLTK.")


Usando 313 stopwords de NLTK.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# 2. Preparación de Datos: Preprocesamiento

Para usar `CountVectorizer` y `TfidfVectorizer`, necesitamos nuestros textos como una lista de strings, donde cada string contiene los tokens (o lemas) relevantes unidos por espacios.

Vamos a reutilizar la función de lematización con spaCy de la clase pasada, ya que generalmente es preferible.

In [5]:
# Función de preprocesamiento (lematización con spaCy)
def preprocesar_texto_para_vectorizar(texto):
  # Limpieza básica
  texto = texto.lower()
  texto = re.sub(r'\d+', '', texto)
  texto = texto.translate(str.maketrans(string.punctuation + '¡¿', ' ' * len(string.punctuation + '¡¿')))
  texto = re.sub(r'\s+', ' ', texto).strip()
  # Lematización y filtrado con spaCy
  doc = nlp(texto)
  lemas = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
  # Unir lemas en un solo string
  return " ".join(lemas)

In [6]:
# Nuestro dataset de ejemplo (puede ser el mismo de las reviews o uno nuevo)
documentos = [
    "El sol brilla y los pájaros cantan alegremente.",
    "La inteligencia artificial transforma el mundo rápidamente.",
    "Me encanta el análisis de datos y el machine learning.",
    "Los pájaros azules cantan en mi ventana.",
    "La inteligencia artificial es un campo fascinante."
]

In [7]:
# Preprocesar todos los documentos
documentos_preprocesados = [preprocesar_texto_para_vectorizar(doc) for doc in documentos]

In [8]:
print("Documentos Originales:")
for doc in documentos:
  print(f"- {doc}")

Documentos Originales:
- El sol brilla y los pájaros cantan alegremente.
- La inteligencia artificial transforma el mundo rápidamente.
- Me encanta el análisis de datos y el machine learning.
- Los pájaros azules cantan en mi ventana.
- La inteligencia artificial es un campo fascinante.


In [9]:
print("\nDocumentos Preprocesados (lemas unidos):")
for doc_proc in documentos_preprocesados:
  print(f"- {doc_proc}")


Documentos Preprocesados (lemas unidos):
- sol brilla pájaro cantar alegremente
- inteligencia artificial transformar mundo rápidamente
- encantar análisis dato machine learning
- pájaro azul cantar ventana
- inteligencia artificial campo fascinante


# 3. Bag-of-Words (BoW) con Scikit-learn (`CountVectorizer`)

**Recordatorio:** BoW representa cada documento como un vector de conteos de palabras. Ignora el orden, solo importa cuántas veces aparece cada palabra del vocabulario total.

Scikit-learn nos facilita esto con `CountVectorizer`.

**Pasos:**
1.  **Instanciar:** Crear un objeto `CountVectorizer`.
2.  **Ajustar (Fit):** Aprender el vocabulario de nuestros documentos preprocesados (`fit()`).
3.  **Transformar (Transform):** Crear la matriz Documento-Término donde cada celda (i, j) tiene el conteo del término j en el documento i (`transform()`).
    *   Podemos hacer `fit()` y `transform()` en un solo paso con `fit_transform()`.

In [10]:
# 1. Instanciar CountVectorizer
vectorizer_bow = CountVectorizer()

In [11]:
# 2. Ajustar y transformar
bow_matrix = vectorizer_bow.fit_transform(documentos_preprocesados)

In [12]:
# Ver el vocabulario aprendido (las columnas de la matriz)
vocabulario = vectorizer_bow.get_feature_names_out()
print("Vocabulario aprendido:")
print(vocabulario)
print(f"\nTamaño del vocabulario: {len(vocabulario)}")

Vocabulario aprendido:
['alegremente' 'análisis' 'artificial' 'azul' 'brilla' 'campo' 'cantar'
 'dato' 'encantar' 'fascinante' 'inteligencia' 'learning' 'machine'
 'mundo' 'pájaro' 'rápidamente' 'sol' 'transformar' 'ventana']

Tamaño del vocabulario: 19


In [13]:
# Ver la matriz BoW (es una matriz dispersa, la convertimos a densa para verla mejor)
# Usamos Pandas para que sea más legible
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=vocabulario, index=[f"Doc_{i+1}" for i in range(len(documentos))])
print("\nMatriz Bag-of-Words (Documento-Término):")
bow_df


Matriz Bag-of-Words (Documento-Término):


,alegremente,análisis,artificial,azul,brilla,campo,cantar,dato,encantar,fascinante,inteligencia,learning,machine,mundo,pájaro,rápidamente,sol,transformar,ventana
Doc_1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0
Doc_2,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
Doc_3,0,1,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0
Doc_4,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
Doc_5,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0


# 4. ¿Por qué BoW no es suficiente? Introducción a TF-IDF

Observen la matriz BoW. Palabras como "cantar" o "inteligencia" o "artificial" aparecen varias veces. Pero, ¿son todas igual de *importantes* para distinguir un documento de otro?

*   "cantar" aparece en Doc_1 y Doc_4. Es algo específico.
*   "inteligencia" y "artificial" aparecen juntas en Doc_2 y Doc_5. Parecen definir un tema.
*   Otras palabras como "pájaro" también aparecen en dos documentos.

BoW solo cuenta. **No diferencia entre palabras comunes (en el corpus) y palabras distintivas.**

**TF-IDF (Term Frequency - Inverse Document Frequency)** intenta solucionar esto.

*   **TF (Frecuencia del Término):** ¿Qué tan frecuente es una palabra *en este documento*? (Similar a BoW, a veces normalizado).
*   **IDF (Frecuencia Inversa de Documento):** ¿Qué tan *rara* es la palabra *en toda la colección* de documentos?
    *   Palabras que aparecen en muchos documentos (como quizás "el", "la" si no las quitáramos) tienen IDF bajo (son poco informativas).
    *   Palabras que aparecen en pocos documentos tienen IDF alto (son más distintivas).
*   **TF-IDF = TF * IDF:** El peso final de una palabra en un documento es alto si es frecuente en ESE documento pero rara en general.

**Resultado:** Una matriz Documento-Término similar a BoW, pero con pesos TF-IDF en lugar de conteos crudos.

# 5. TF-IDF con Scikit-learn (`TfidfVectorizer`)

Scikit-learn también tiene una clase para esto: `TfidfVectorizer`. Funciona de manera muy similar a `CountVectorizer`.

**Pasos:**
1.  **Instanciar:** Crear un objeto `TfidfVectorizer`. Podemos pasarle parámetros como `min_df`, `max_df`, `ngram_range` si queremos.
2.  **Ajustar y Transformar:** Aprender el vocabulario y calcular los pesos IDF del corpus, y luego transformar los documentos en la matriz TF-IDF (`fit_transform()`).

In [14]:
# 1. Instanciar TfidfVectorizer
# Podemos usar los mismos parámetros que CountVectorizer si quisiéramos (ej: min_df, max_df)
vectorizer_tfidf = TfidfVectorizer()

In [15]:
# 2. Ajustar y transformar (usando los mismos documentos preprocesados)
tfidf_matrix = vectorizer_tfidf.fit_transform(documentos_preprocesados)

In [16]:
# El vocabulario debería ser el mismo si no usamos parámetros diferentes
# ¡Ojo! El orden de las columnas puede cambiar entre vectorizadores si no se fija!
# Para comparar fácil, usemos el vocabulario del TF-IDF vectorizer para ambos DFs
vocabulario_tfidf = vectorizer_tfidf.get_feature_names_out()
print("Vocabulario (TF-IDF):")
print(vocabulario_tfidf)

Vocabulario (TF-IDF):
['alegremente' 'análisis' 'artificial' 'azul' 'brilla' 'campo' 'cantar'
 'dato' 'encantar' 'fascinante' 'inteligencia' 'learning' 'machine'
 'mundo' 'pájaro' 'rápidamente' 'sol' 'transformar' 'ventana']


In [17]:
# Crear DataFrame para TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vocabulario_tfidf, index=[f"Doc_{i+1}" for i in range(len(documentos))])
print("\nMatriz TF-IDF (Documento-Término):")
# Redondear para mejor visualización
print(tfidf_df.round(3))


Matriz TF-IDF (Documento-Término):
       alegremente  análisis  artificial  azul  brilla  campo  cantar   dato  \
Doc_1        0.482     0.000       0.000  0.00   0.482   0.00   0.389  0.000   
Doc_2        0.000     0.000       0.389  0.00   0.000   0.00   0.000  0.000   
Doc_3        0.000     0.447       0.000  0.00   0.000   0.00   0.000  0.447   
Doc_4        0.000     0.000       0.000  0.55   0.000   0.00   0.444  0.000   
Doc_5        0.000     0.000       0.444  0.00   0.000   0.55   0.000  0.000   

       encantar  fascinante  inteligencia  learning  machine  mundo  pájaro  \
Doc_1     0.000        0.00         0.000     0.000    0.000  0.000   0.389   
Doc_2     0.000        0.00         0.389     0.000    0.000  0.482   0.000   
Doc_3     0.447        0.00         0.000     0.447    0.447  0.000   0.000   
Doc_4     0.000        0.00         0.000     0.000    0.000  0.000   0.444   
Doc_5     0.000        0.55         0.444     0.000    0.000  0.000   0.000   

       r

In [18]:
# Nota: TfidfVectorizer puede devolver un vocabulario diferente o en orden diferente
# si se usan parámetros como min_df/max_df.
# Para una comparación directa, podemos re-instanciar CountVectorizer con el
# vocabulario aprendido por TfidfVectorizer
vectorizer_bow_for_compare = CountVectorizer(vocabulary=vocabulario_tfidf)
bow_matrix_for_compare = vectorizer_bow_for_compare.fit_transform(documentos_preprocesados)
bow_df_for_compare = pd.DataFrame(bow_matrix_for_compare.toarray(), columns=vocabulario_tfidf, index=[f"Doc_{i+1}" for i in range(len(documentos))])

# 6. Comparando BoW y TF-IDF

Ahora veamos las dos matrices juntas (usando el mismo vocabulario y orden de columnas para que sea fácil comparar).

Fíjense en cómo cambian los valores para algunas palabras clave.

In [19]:
print("--- Comparación BoW vs TF-IDF ---")

print("\nMatriz Bag-of-Words (Revisada con vocabulario TF-IDF):")
print(bow_df_for_compare)

print("\nMatriz TF-IDF:")
print(tfidf_df.round(3))

--- Comparación BoW vs TF-IDF ---

Matriz Bag-of-Words (Revisada con vocabulario TF-IDF):
       alegremente  análisis  artificial  azul  brilla  campo  cantar  dato  \
Doc_1            1         0           0     0       1      0       1     0   
Doc_2            0         0           1     0       0      0       0     0   
Doc_3            0         1           0     0       0      0       0     1   
Doc_4            0         0           0     1       0      0       1     0   
Doc_5            0         0           1     0       0      1       0     0   

       encantar  fascinante  inteligencia  learning  machine  mundo  pájaro  \
Doc_1         0           0             0         0        0      0       1   
Doc_2         0           0             1         0        0      1       0   
Doc_3         1           0             0         1        1      0       0   
Doc_4         0           0             0         0        0      0       1   
Doc_5         0           1             

In [20]:
print("\n--- Análisis ---")
# Ejemplo: Palabra 'pájaro'
print("\nPalabra: 'pájaro'")
print("  BoW  : ", bow_df_for_compare['pájaro'].values)
print("  TF-IDF: ", tfidf_df['pájaro'].round(3).values)
# Aparece en Doc 1 y 4. Tiene un peso TF-IDF > 0 pero no el máximo, porque aparece en 2/5 documentos.


--- Análisis ---

Palabra: 'pájaro'
  BoW  :  [1 0 0 1 0]
  TF-IDF:  [0.389 0.    0.    0.444 0.   ]


In [21]:
# Ejemplo: Palabra 'sol'
print("\nPalabra: 'sol'")
print("  BoW  : ", bow_df_for_compare['sol'].values)
print("  TF-IDF: ", tfidf_df['sol'].round(3).values)
# Solo aparece en Doc 1. ¡Tiene un peso TF-IDF alto en ese documento! Es distintiva.


Palabra: 'sol'
  BoW  :  [1 0 0 0 0]
  TF-IDF:  [0.482 0.    0.    0.    0.   ]


In [22]:
# Ejemplo: Palabra 'inteligencia'
print("\nPalabra: 'inteligencia'")
print("  BoW  : ", bow_df_for_compare['inteligencia'].values)
print("  TF-IDF: ", tfidf_df['inteligencia'].round(3).values)
# Aparece en Doc 2 y 5. Tiene peso TF-IDF, pero menor que 'sol', porque es menos rara.


Palabra: 'inteligencia'
  BoW  :  [0 1 0 0 1]
  TF-IDF:  [0.    0.389 0.    0.    0.444]


In [23]:
# Ejemplo: Palabra 'artificial'
print("\nPalabra: 'artificial'")
print("  BoW  : ", bow_df_for_compare['artificial'].values)
print("  TF-IDF: ", tfidf_df['artificial'].round(3).values)
# Idem 'inteligencia'.


Palabra: 'artificial'
  BoW  :  [0 1 0 0 1]
  TF-IDF:  [0.    0.389 0.    0.    0.444]


**Conclusiones de la Comparación:**

*   BoW da igual importancia (si aparecen 1 vez) a "sol" y "pájaro" en el Doc 1.
*   TF-IDF le da más peso a "sol" en el Doc 1 porque "sol" es más rara en todo el corpus (solo aparece ahí) que "pájaro" (aparece en Doc 1 y 4).
*   TF-IDF captura mejor la "importancia relativa" o "distintividad" de las palabras.
*   Las palabras que aparecen en muchos documentos (si hubiera) tendrían un peso TF-IDF muy bajo.

**Ventajas de TF-IDF sobre BoW:** Generalmente produce mejores resultados en tareas como clasificación de texto, clustering y búsqueda de información porque pondera las palabras de forma más inteligente.

**Limitaciones (que aún persisten):**
*   **Ignora el orden de las palabras:** "hombre muerde perro" y "perro muerde hombre" tendrían representaciones muy similares.
*   **No captura sinonimia:** "auto" y "coche" son dimensiones diferentes.
*   **Dimensionalidad alta:** Seguimos teniendo tantas columnas como palabras únicas en el vocabulario.

# 7. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:**

Usando las `reviews` y las funciones de preprocesamiento de clases previas (o volviendo a procesarlas ahora):
1.  Asegúrate de tener la lista de `reviews_preprocesadas` (cada elemento es un string con los lemas unidos por espacios). Si no la tenés, generála usando la función `preprocesar_texto_para_vectorizar` sobre las `reviews` originales.
2.  **Crear Matriz BoW:**
    *   Instancia un `CountVectorizer`.
    *   Aplícalo a las `reviews_preprocesadas` usando `fit_transform()`.
    *   Obtén el vocabulario (`get_feature_names_out()`).
    *   Crea un DataFrame de Pandas para visualizar la matriz BoW.
3.  **Crear Matriz TF-IDF:**
    *   Instancia un `TfidfVectorizer`.
    *   Aplícalo a las **mismas** `reviews_preprocesadas` usando `fit_transform()`.
    *   **Importante:** Para comparar fácil, puedes pasarle el vocabulario aprendido por el CountVectorizer al TfidfVectorizer usando el argumento `vocabulary=`. O viceversa. La idea es que ambas matrices usen las mismas columnas en el mismo orden.
    *   Crea un DataFrame de Pandas para visualizar la matriz TF-IDF (redondea los valores a 3 decimales).
4.  **Analizar:**
    *   Imprime ambas matrices.
    *   Elige una o dos reviews. ¿Qué palabras tienen los pesos más altos en TF-IDF para esa review? ¿Coincide con lo que esperarías que sean las palabras clave de esa review?
    *   Busca alguna palabra que tenga un conteo > 0 en BoW pero un peso TF-IDF relativamente bajo. ¿Por qué podría ser? (Pista: ¿aparece en muchas reviews?).

In [24]:
# Dataset (el mismo del martes)
reviews = [
    "Una película emocionante con actuaciones brillantes. ¡Me encantó!",
    "Muy aburrida y lenta. El guión era predecible y los actores no convencían.",
    "Los efectos especiales fueron impresionantes, pero la historia dejaba mucho que desear.",
    "¡Qué gran comedia! Me reí sin parar durante toda la película.",
    "Un documental necesario que aborda temas importantes con profundidad y sensibilidad."
]

# 8. Brainstorming

Hemos visto BoW y TF-IDF. Son pasos importantes, pero ¿son suficientes?

**¿Cómo podemos representar el texto de manera que se preserve la información relevante y se minimice el ruido?**

*   ¿Qué información crucial **pierden** BoW y TF-IDF? (¡El orden de las palabras! La semántica, la relación entre palabras).
*   "El rey mató a la reina" vs "La reina mató al rey". ¿BoW/TF-IDF las distinguirían bien? (No mucho).
*   ¿Cómo podríamos capturar que "coche" y "auto" significan casi lo mismo? (BoW/TF-IDF las tratan como totalmente diferentes).
*   ¿Qué pasa con la **dimensionalidad**? Si tenemos 50,000 palabras únicas, ¡nuestros vectores tienen 50,000 dimensiones! ¿Es eficiente?
*   ¿Cómo afectan nuestras decisiones de preprocesamiento (stemming vs lematización, quitar o no ciertas palabras) a estas representaciones?

**Próximos pasos (anticipo):**
*   **Word Embeddings (Word2Vec, GloVe, FastText):** Representar palabras como vectores densos (no dispersos como BoW/TF-IDF) en un espacio donde palabras semánticamente similares estén cerca. ¡Capturan significado!
*   **Modelos Secuenciales (RNN, LSTM, GRU):** Redes neuronales diseñadas para procesar secuencias, teniendo en cuenta el orden de las palabras.
*   **Transformers (BERT, GPT):** Arquitecturas más modernas que usan mecanismos de "atención" para entender el contexto de cada palabra en la oración.

**(Discusión en grupo)**